In [1]:
import cv2 as cv
import numpy as np

In [2]:
img_fn = ["data/2/img-1.tif", "data/2/img-2.tif", "data/2/img-3.tif", "data/2/img-4.tif", "data/2/img-5.tif", "data/2/img-6.tif", "data/2/img-7.tif", "data/2/img-8.tif", "data/2/img-9.tif"]
img_list = [cv.imread(fn) for fn in img_fn]
exposure_times = np.array([(1/8000), (1/4000), (1/2000), (1/1000), (1/500), (1/250), (1/125), (1/60), (1/30)], dtype=np.float32)

### Register images

In [3]:
gray = [cv.cvtColor(img, cv.COLOR_RGB2GRAY) for img in img_list]
norm = [cv.equalizeHist(test) for test in gray]
img1 = norm[0]

output = [img_list[0]]
for i in range(1, len(norm)):
    print(i)
    img2 = norm[i]

    detector = cv.AKAZE_create()
    kp1, des1 = detector.detectAndCompute(img1, None)
    kp2, des2 = detector.detectAndCompute(img2, None)

    bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck = True)
    matches = bf.match(des1, des2)

    matches = sorted(matches, key = lambda x: x.distance)

    num_good_matches = int(len(matches) * 0.75)
    matches = matches[:max(num_good_matches, 10)]

    pts1 = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 2)
    pts2 = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 2)

    transformation_matrix, inliers = cv.findHomography(
                pts2, pts1, method=cv.RANSAC, ransacReprojThreshold=3.0, confidence=0.99
            )

    aligned_img = cv.warpPerspective(
                img_list[i], 
                transformation_matrix, 
                (img1.shape[1], img1.shape[0]),
                flags=cv.INTER_LINEAR
            )
    output += [aligned_img]

1


KeyboardInterrupt: 

In [ ]:
merge_debevec = cv.createMergeDebevec()
hdr_debevec = merge_debevec.process(output, times=exposure_times.copy())

In [ ]:
tonemap1 = cv.createTonemap(gamma=2.2)
res_debevec = tonemap1.process(hdr_debevec.copy())

In [ ]:
res_debevec_8bit = np.clip(res_debevec*255, 0, 255).astype('uint8')

C:\Users\Alex\AppData\Local\Temp\ipykernel_37012\2826973509.py:1: RuntimeWarning: invalid value encountered in cast
  res_debevec_8bit = np.clip(res_debevec*255, 0, 255).astype('uint8')


In [ ]:
cv.imwrite("ldr_debevec.jpg",
 res_debevec_8bit)

True

In [ ]:
for i in range(len(output)):
    cv.imwrite(str(i) + ".png", output[i])